In [1]:
import numpy as np
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister, AncillaRegister
from qiskit import Aer
from qiskit.opflow import X, Z, I, MatrixEvolution, Y
from qiskit.quantum_info.operators import Operator, Pauli
from qiskit.circuit import Parameter
from qiskit import transpile
from scipy import linalg
from scipy.special import binom
import matplotlib.pyplot as plt
from qutip import *
import itertools as it
import copy
import stomp_functions as stf
from qiskit.quantum_info import random_clifford
import time
from qiskit.providers.fake_provider import FakeBelem
from qiskit.circuit.library import PauliEvolutionGate

In [2]:
# Set font size of plot elements
SMALL_SIZE = 12
MEDIUM_SIZE = 14
BIGGER_SIZE = 18

plt.rc('font', size=SMALL_SIZE)          # controls default text sizes
plt.rc('axes', titlesize=SMALL_SIZE)     # fontsize of the axes title
plt.rc('axes', labelsize=MEDIUM_SIZE)    # fontsize of the x and y labels
plt.rc('xtick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('ytick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('legend', fontsize=SMALL_SIZE)    # legend fontsize
plt.rc('figure', titlesize=BIGGER_SIZE)  # fontsize of the figure title

In [3]:
backend = FakeBelem()
#backend = Aer.get_backend("aer_simulator")
num_shots = 1*8192

# Set up parameters for unitary
beta = 1
num_steps = 500
betas, dt = np.linspace(0, beta, num_steps, retstep=True)

In [4]:
# Define parameters
D = -2
γ = 0.5
N = 2

In [5]:
# Define Hamiltonian
H_qis = D / 2 * ((I ^ I) + (Z ^ Z)) + γ / 2 * ((X ^ X) - (Y ^ Y))

In [6]:
# Check that Eq. 9 for the evolution operator matches what qiskit yields
U_qis = (float(dt) * H_qis).exp_i().to_matrix()

coeff = np.exp(-1j * D / 2 * dt)
A = (D / 2 * dt * (Z ^ Z)).exp_i().to_matrix()
B = (γ / 2 * dt * (X ^ X)).exp_i().to_matrix()
C = (-γ / 2 * dt * (Y ^ Y)).exp_i().to_matrix()
U = coeff * A @ B @ C

print(np.allclose(U, U_qis))

True


In [7]:
# Get eigenenergies of H
E_qis, V_qis = np.linalg.eigh(H_qis.to_matrix())

In [8]:
# Define initial wavefunction
init_wf = np.array([1, 0, 0, 0])

# Create circuit registers
qr = QuantumRegister(N)
qar = AncillaRegister(1)
cr = ClassicalRegister(1)
cliff = random_clifford(N, seed=5)
init_wf = cliff.to_matrix() @ init_wf

In [9]:
# Get circuits for real and imaginary Hadamard estimation with no observable
he_circs_re = stf.had_est_barr(qr, qar, cr, H_qis, None, num_steps, 
                                      dt, cliff)
he_circs_im = stf.im_had_est_barr(qr, qar, cr, H_qis, None, num_steps, 
                                      dt, cliff)

In [10]:
# Get the overlap lists for the circuits with no observables
ovlps_r = stf.get_ovlps(he_circs_re, backend, num_shots)
ovlps_i = stf.get_ovlps(he_circs_im, backend, num_shots)
ovlps = ovlps_r + 1j * ovlps_i

In [11]:
# Now that we have the overlaps, we want to calculate the expectation value of H at each step
pauli_H = stf.pauli_string_decomp(H_qis, N)

In [12]:
# Now perform hadmard estimation on each string
H_circs_r = {}
H_circs_i = {}
for key in pauli_H:
    if abs(pauli_H[key]) != 0:
        H_circs_r[key] = stf.had_est_barr(qr, qar, cr, H_qis, Pauli(key), num_steps, dt,
                                                 cliff)
        
        H_circs_i[key] = stf.im_had_est_barr(qr, qar, cr, H_qis, Pauli(key), num_steps,
                                                                   dt, cliff)

In [13]:
# Get real part of H expectations
H_ovlps_re = 0
for key in H_circs_r:
    temp = stf.get_ovlps(H_circs_r[key], backend, num_shots)
    H_ovlps_re += pauli_H[key] * temp / np.sqrt(2 ** (2*N))

In [ ]:
# Get imag part of H expectations
H_ovlps_im = 0
for key in H_circs_i:
    temp = stf.get_ovlps(H_circs_i[key], backend, num_shots)
    H_ovlps_im += 1j * pauli_H[key] * temp / np.sqrt(2 ** (2*N))

In [ ]:
# Get total H expectation
total_H_ovlp = H_ovlps_re + H_ovlps_im

In [ ]:
# Classical calculation
class_ovlp, class_E_ovlp = stf.classical_calc(np.array(init_wf), H_qis.to_matrix(),
                                              H_qis.to_matrix(), num_steps, dt)

In [ ]:
plt.figure(1)
plt.plot(ovlps.real, 'o-', label='Aer')
plt.plot(class_ovlp.real, 'kx-', label='Classical')
plt.ylabel("Re(Ovlp)")
plt.xlabel("steps")
plt.legend(numpoints=1)

plt.figure(2)
plt.plot(ovlps.imag, 'o-', label='Aer')
plt.plot(class_ovlp.imag, 'kx-', label='Classical')
plt.ylabel("Im(Ovlp)")
plt.xlabel("steps")
plt.legend(numpoints=1)

In [ ]:
plt.figure(1)
plt.plot(total_H_ovlp.real, 'o-', label='Aer')
plt.plot(class_E_ovlp.real, 'kx-', label='Classical')
plt.xlabel("Steps")
plt.ylabel("Re(E)")
plt.legend(numpoints=1)

plt.figure(2)
plt.plot(total_H_ovlp.imag, 'o-', label='Aer')
plt.plot(class_E_ovlp.imag, 'kx-', label='Classical')
plt.xlabel("Steps")
plt.ylabel("Im(E)")
plt.legend(numpoints=1)

In [ ]:
# Sweep over lambdas to find good values for convergence
λ = np.linspace(-1.5 * np.abs(E_qis[0]), 0.9 * np.abs(E_qis[0]), 1000)
calc_eng = [stf.alt_partition_calc(ovlps, total_H_ovlp, num_steps, _, dt)[1][-1]
           for _ in λ]

In [ ]:
for _ in E_qis:
    plt.plot(λ / np.abs(E_qis[0]), _ * np.ones(λ.shape[0]) + λ, 'k-.')
plt.plot(λ / np.abs(E_qis[0]), calc_eng)
plt.xlabel("$\\lambda / E_0$")
plt.ylabel("E")

In [ ]:
λs = np.array([-1.5, -0.5, 0.5]) * E_qis[0]
l = 0
for _ in zip(λs, E_qis):
    plt.plot(betas[2::2], stf.alt_partition_calc(ovlps, total_H_ovlp, num_steps,
                                                -_[0], dt)[1][1:]+_[0],
            label='$\\lambda=E_' + str(l) + '$')
    l += 1
    plt.plot(betas, _[1] * np.ones(len(betas)), '-.')

plt.xlabel("$\\beta$")
plt.ylabel("E")
plt.legend()
plt.savefig("gnatenkoH_belem.png", format='png', dpi=300)

In [ ]:
trans_re = transpile(he_circs_re, backend)
trans_im = transpile(he_circs_im, backend)
re_depths = [_.depth() for _ in trans_re]
im_depths = [_.depth() for _ in trans_re]

In [ ]:
plt.plot(re_depths, label='Real')
plt.plot(im_depths, label='Imag')
plt.xlabel("Number of Steps")
plt.ylabel("Circuit Depth")